In [1]:
import cml.data_v1 as cmldata

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

23/09/23 22:21:55 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/23 22:21:55 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/23 22:21:56 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/23 

+--------------------+
|           namespace|
+--------------------+
|         01_car_data|
|           01_car_dw|
|              adb101|
|            airlines|
|        airlines_csv|
|    airlines_iceberg|
|airlines_iceberg_...|
|      airlines_mjain|
|          airquality|
|                ajvp|
|          atlas_demo|
|            bankdemo|
|          bca_jps_l0|
|        cde_workshop|
|cde_workshop_pdef...|
|             cdedemo|
|        cdp_overview|
|      ceht_open_data|
|        ceht_scratch|
| ceht_transportation|
+--------------------+
only showing top 20 rows



In [2]:
spark.sparkContext.getConf().getAll()

23/09/23 22:22:06 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/09/23 22:22:06 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


[('spark.eventLog.enabled', 'true'),
 ('spark.network.crypto.enabled', 'true'),
 ('spark.sql.hive.hwc.execution.mode', 'spark'),
 ('spark.jars',
  '/opt/spark/optional-lib/hive-warehouse-connector-assembly.jar,/opt/spark/optional-lib/iceberg-hive-runtime.jar,/opt/spark/optional-lib/iceberg-spark-runtime.jar'),
 ('spark.kerberos.renewal.credentials', 'ccache'),
 ('spark.sql.catalog.spark_catalog',
  'org.apache.iceberg.spark.SparkSessionCatalog'),
 ('spark.dynamicAllocation.maxExecutors', '49'),
 ('spark.eventLog.dir', 'file:///sparkeventlogs'),
 ('spark.hadoop.yarn.resourcemanager.principal', 'pauldefusco'),
 ('spark.kubernetes.driver.annotation.cluster-autoscaler.kubernetes.io/safe-to-evict',
  'false'),
 ('spark.ui.port', '20049'),
 ('spark.kubernetes.driver.pod.name', 'a8r5dpdadtfatd8k'),
 ('spark.driver.bindAddress', '100.100.175.202'),
 ('spark.yarn.access.hadoopFileSystems',
  's3a://go01-demo/warehouse/tablespace/external/hive'),
 ('spark.sql.extensions',
  'com.qubole.spark.hiv

In [3]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [4]:
# Create a new database
spark.sql("CREATE DATABASE IF NOT EXISTS spark_catalog.lakehouse")
spark.sql("USE spark_catalog.lakehouse")

DataFrame[]

In [7]:
spark.sql("DROP TABLE IF EXISTS lakehouse.coffees_table_part_evol PURGE")

DataFrame[]

In [8]:
spark.sql("CREATE TABLE IF NOT EXISTS coffees_table_part_evol (coffee_id BIGINT, coffee_size STRING, coffee_sale_ts TIMESTAMP)\
          USING ICEBERG\
          PARTITIONED BY (months(coffee_sale_ts))\
          TBLPROPERTIES ('write.delete.mode'='copy-on-write',\
                          'write.update.mode'='copy-on-write',\
                          'write.merge.mode'='copy-on-write',\
                          'format-version' = '2')")

DataFrame[]

In [9]:
spark.sql("INSERT INTO lakehouse.coffees_table_part_evol\
    VALUES (2, 'tall', cast(date_format('2023-08-01 11:10:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
    (3, 'venti', cast(date_format('2023-04-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
    (4, 'venti', cast(date_format('2023-07-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
    (5, 'grande', cast(date_format('2023-07-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
    (6, 'grande', cast(date_format('2023-07-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
    (7, 'venti', cast(date_format('2023-05-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
    (8, 'grande', cast(date_format('2023-04-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
    (9, 'tall', cast(date_format('2023-05-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
    (10, 'tall', cast(date_format('2023-05-01 12:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp))")

DataFrame[]

Spark partitioning is a way to split the data into multiple partitions so that you can execute transformations on multiple partitions in parallel which allows completing the job faster. You can also write partitioned data into a file system (multiple sub-directories) for faster reads by downstream systems.

Iceberg provides a method to look into the current partitions via Spark SQL

In [10]:
print("TABLE PARTITIONS BEFORE ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM lakehouse.coffees_table_part_evol.partitions").show()

TABLE PARTITIONS BEFORE ALTER PARTITION STATEMENT: 
+---------+------------+----------+-------+
|partition|record_count|file_count|spec_id|
+---------+------------+----------+-------+
|    {642}|           3|         1|      0|
|    {643}|           1|         1|      0|
|    {639}|           2|         1|      0|
|    {640}|           3|         1|      0|
+---------+------------+----------+-------+



To migrate from daily to hourly partitioning with transforms, it is not necessary to drop the daily partition field. Keeping the field ensures existing metadata table queries continue to work.

In [14]:
print("REPLACE PARTITION MONTHS(COFFEE SALE TS) WITH DAY (COFFEE SALE TS): ")
#print("ALTER TABLE spark_catalog.lakehouse.coffees_table_part_evol ADD PARTITION FIELD months(event_ts)")
#spark.sql("ALTER TABLE spark_catalog.lakehouse.coffees_table_part_evol ADD PARTITION FIELD months(event_ts)")
#spark.sql("ALTER TABLE spark_catalog.lakehouse.part_evol_tbl REPLACE PARTITION FIELD hours(dob) WITH state")
#spark.sql("ALTER TABLE prod.db.sample ADD PARTITION FIELD month")

spark.sql("ALTER TABLE lakehouse.coffees_table_part_evol REPLACE PARTITION FIELD months(coffee_sale_ts) WITH days(coffee_sale_ts)")

#ALTER TABLE spark_catalog.lakehouse.part_evol_tbl ADD PARTITION FIELD days(event_ts)

REPLACE PARTITION MONTHS(COFFEE SALE TS) WITH DAY (COFFEE SALE TS): 


DataFrame[]

In [16]:
print("TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.coffees_table_part_evol.PARTITIONS").show()

TABLE PARTITIONS AFTER ALTER PARTITION STATEMENT: 
+-----------+------------+----------+-------+
|  partition|record_count|file_count|spec_id|
+-----------+------------+----------+-------+
|{639, null}|           2|         1|      0|
|{643, null}|           1|         1|      0|
|{642, null}|           3|         1|      0|
|{640, null}|           3|         1|      0|
+-----------+------------+----------+-------+



#### Notice partitioning is unchaged. This is because we haven't added any data.

In [17]:
appendDf = spark.sql("SELECT * FROM spark_catalog.lakehouse.coffees_table_part_evol TABLESAMPLE (50 PERCENT)")

In [19]:
appendDf.sortWithinPartitions("coffee_size").writeTo("spark_catalog.lakehouse.coffees_table_part_evol").using("iceberg").append() #.append()#replace()#overwritePartitions()#create()

In [20]:
print("TABLE PARTITIONS AFTER APPEND STATEMENT: ")
spark.sql("SELECT * FROM spark_catalog.lakehouse.coffees_table_part_evol.PARTITIONS").show()

TABLE PARTITIONS AFTER APPEND STATEMENT: 
+------------------+------------+----------+-------+
|         partition|record_count|file_count|spec_id|
+------------------+------------+----------+-------+
|       {639, null}|           2|         1|      0|
|       {643, null}|           1|         1|      0|
|       {642, null}|           3|         1|      0|
|       {640, null}|           3|         1|      0|
|{null, 2023-05-01}|           1|         1|      1|
|{null, 2023-04-01}|           1|         1|      1|
+------------------+------------+----------+-------+



Adding a partition field is a metadata operation and does not change any of the existing table data. New data will be written with the new partitioning, but existing data will remain in the old partition layout. Old data files will have null values for the new partition fields in metadata tables.